In [1]:
import torch
print(torch.__version__)

2.0.1+cu117


In [2]:
# %load eval.py
from argparse import ArgumentParser

from pytorch_lightning import seed_everything, Trainer

from dataset.mpii_face_gaze_dataset import get_dataloaders
from train import Model

import numpy as np
import torch
from utils import calc_angle_error,compute_bias_eq3,calc_angle_error_for_eval
from sklearn.linear_model import LinearRegression

import random

INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.7 (you have 1.4.11). Upgrade using: pip install --upgrade albumentations


In [3]:
def select_MGTC_calibration_samples(test_set, s=9, grid_bins=4):
    """
    從整個測試集中選擇來自不同 gaze 方向的 S 張樣本，用於 MGTC 模擬。
    
    :param test_set: 測試資料集（list of dict）
    :param s: 需要抽取的樣本數
    :param grid_bins: pitch/yaw 空間分割格數（預設 8x8）
    :return: List of dict（被選出的 sample）
    """

    # 每個 sample 的 pitch/yaw 轉成格子 key
    bins = {}
    for idx, item in enumerate(test_set):
        pitch = float(item['gaze_pitch'])
        yaw = float(item['gaze_yaw'])

        pitch_bin = int((pitch + np.pi / 2) / (np.pi / grid_bins))
        yaw_bin = int((yaw + np.pi / 2) / (np.pi / grid_bins))
        key = (pitch_bin, yaw_bin)

        if key not in bins:
            bins[key] = []
        bins[key].append(item)

    # 隨機從不同格子抽樣，盡量挑出 s 張圖
    keys = list(bins.keys())
    random.shuffle(keys)

    selected_samples = []
    for key in keys:
        candidates = bins[key]
        if candidates:
            chosen = random.choice(candidates)
            selected_samples.append(chosen)
        if len(selected_samples) >= s:
            break

    if len(selected_samples) < s:
        print(f"⚠️ MGTC 抽樣失敗，只取得 {len(selected_samples)} 張樣本")

    return selected_samples


In [4]:
def evaluate_with_eq3_repeat(model, test_set, s=9, repeat_times=10, target_pitch=0.0, target_yaw=0.0, device='cuda', use_mgtc=False):
    """
    重複多次隨機挑選 S 張校正圖，模擬作者論文中的 ± 誤差統計

    :param model: 已訓練好的模型
    :param test_set: 測試資料集（list of dict）
    :param s: 每次校正圖數量 S
    :param repeat_times: 重複幾次校正試驗
    :param target_pitch: 校正注視點 pitch（預設 0.0）
    :param target_yaw: 校正注視點 yaw（預設 0.0）
    :param device: 預設 'cuda'
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    print(f"🔍 校正模式：{'MGTC（多方向）' if use_mgtc else 'SGTC（單方向）'}")

    all_mean_errors = []

    for trial in range(repeat_times):
        # Step 1: 找出符合目標注視點的樣本(STGC/MTGC)

        if use_mgtc:
            calibration_samples = select_MGTC_calibration_samples(test_set, s=s)
            if len(calibration_samples) < s:
                print(f"⚠️ 第 {trial+1} 次 MGTC 僅取得 {len(calibration_samples)} 張，將隨機補滿")
                # 這裡比對 file_name 避免 tensor 無法比較的錯誤
                calibration_names = set(item['file_name'] for item in calibration_samples)
                remaining = [item for item in test_set if item['file_name'] not in calibration_names]
                extra = random.sample(remaining, s - len(calibration_samples))
                calibration_samples += extra
        else:
            calibration_samples = [
                item for item in test_set
                if abs(item['gaze_pitch'] - target_pitch) < 0.1 and abs(item['gaze_yaw'] - target_yaw) < 0.1
            ]
            if len(calibration_samples) < s:
                print(f"❌ 第 {trial+1} 次校正失敗，符合條件的圖像不足 {s} 張")
                continue

        selected = random.sample(calibration_samples, s)
        cal_names = set(item['file_name'] for item in selected)

        # Step 2: 計算偏移量
        b_hat = compute_bias_eq3(model, selected, device)

        # Step 3: 測試其餘圖片的 angular error
        preds, labels = [], []

        model.eval()
        with torch.no_grad():
            for item in test_set:
                if item['file_name'] in cal_names:
                    continue

                full_face = item['full_face_image'].unsqueeze(0).to(device)
                right_eye = item['right_eye_image'].unsqueeze(0).to(device)
                left_eye = item['left_eye_image'].unsqueeze(0).to(device)

                t_hat = model.get_subject_independent_output(full_face, right_eye, left_eye)
                g_hat = t_hat + b_hat.to(t_hat.device)

                preds.append(g_hat.squeeze(0).cpu())
                labels.append(torch.tensor([
                    float(item['gaze_pitch']),
                    float(item['gaze_yaw'])
                ], dtype=torch.float32))

        # Step 4: 計算一次 mean angular error
        preds = torch.stack(preds)
        labels = torch.stack(labels)
        error = calc_angle_error_for_eval(labels, preds).mean().item()
        all_mean_errors.append(error)

    # Step 5: 最終 ± 統計
    if len(all_mean_errors) == 0:
        print("⚠️ 所有試驗都失敗，無法產生 ± 統計")
        return None

    mean_err = np.mean(all_mean_errors)
    std_err = np.std(all_mean_errors)
    print(f"📐 Angular Error (S={s}, N={repeat_times}): {mean_err:.2f} ± {std_err:.2f} degrees")

    return mean_err, std_err


In [5]:
# ✅ 新增這個輔助函數
def calculate_error_distribution(error_list):
    """
    輸入每張圖的 angular error list，輸出 mean ± std 並印出
    """
    mean_angle = np.mean(error_list)
    std_angle = np.std(error_list)
    print(f"📐 Angular Error: {mean_angle:.2f} ± {std_angle:.2f} degrees")
    return mean_angle, std_angle
# ✅ 主函數
def evaluate_with_eq3(model, test_set, device='cuda', s=9, target_pitch=0.0, target_yaw=0.0):
    """
    用 Eq.3 進行校正後測試，模擬 SGTC (Single Gaze Target Calibration)
    """
    # Step 1: 選擇校正圖像
    calibration_samples = [
        item for item in test_set
        if abs(item['gaze_pitch'] - target_pitch) < 0.1 and abs(item['gaze_yaw'] - target_yaw) < 0.1
    ][:s]

    if len(calibration_samples) < s:
        print(f"⚠️ 校正圖數不足 {s} 張，只找到 {len(calibration_samples)} 張")
        return None

    model = model.to(device)
    calibration_filenames = set(item['file_name'] for item in calibration_samples)

    # Step 2: 計算 Eq.3 偏移 b̂
    b_hat = compute_bias_eq3(model, calibration_samples, device)

    # Step 3: 使用偏移進行預測
    preds = []
    labels = []

    ### ✅ 修改點 START：記錄每張 angular error
    angle_errors = []
    ### ✅ 修改點 END

    model.eval()
    with torch.no_grad():
        for item in test_set:
            if item['file_name'] in calibration_filenames:
                continue

            full_face = item['full_face_image'].unsqueeze(0).to(device)
            right_eye = item['right_eye_image'].unsqueeze(0).to(device)
            left_eye = item['left_eye_image'].unsqueeze(0).to(device)

            t_hat = model.get_subject_independent_output(full_face, right_eye, left_eye)
            g_hat = t_hat + b_hat.to(t_hat.device)

            preds.append(g_hat.squeeze(0).cpu())
            label_tensor = torch.tensor([
                float(item['gaze_pitch']),
                float(item['gaze_yaw'])
            ], device=device, dtype=torch.float32)
            labels.append(label_tensor)

            ### ✅ 修改點 START：單張 angular error 記錄
            angle = calc_angle_error(label_tensor.unsqueeze(0), g_hat.squeeze(0).unsqueeze(0))
            angle_errors.append(angle.item())
            ### ✅ 修改點 END

    # Step 4: 輸出 ± 統計
    calculate_error_distribution(angle_errors)

    return np.mean(angle_errors)

In [6]:
def simulate_calibration_lr(model, dataset, S=1, device='cuda'):
    """
    使用 Linear Regression 做視線偏移校正
    """
    model = model.to(device)
    model.eval()

    with torch.no_grad():
        all_data = []
        for i in range(len(dataset)):
            item = dataset[i]
            data = {
                'left_eye': item['left_eye_image'].unsqueeze(0).to(device),
                'right_eye': item['right_eye_image'].unsqueeze(0).to(device),
                'face': item['full_face_image'].unsqueeze(0).to(device),
                'label': torch.tensor(
                    [float(item['gaze_pitch']), float(item['gaze_yaw'])],
                    dtype=torch.float32
                ).to(device),
                'person_idx': torch.tensor([[item['person_idx']]]).to(device)
            }
            all_data.append(data)

        # 選 S 張圖作為校正資料
        indices = np.random.choice(len(all_data), size=S, replace=False)
        calibration_set = [all_data[i] for i in indices]
        test_set = [all_data[i] for i in range(len(all_data)) if i not in indices]

        # 擷取 calibration 資料的 pred & label
        preds, labels = [], []
        for d in calibration_set:
            pred = model(
                d['person_idx'], d['face'], d['right_eye'], d['left_eye']
            ).squeeze(0).cpu().numpy()
            label = d['label'].cpu().numpy()
            preds.append(pred)
            labels.append(label)

        # 執行 Linear Regression 擬合 (pred -> label)
        reg = LinearRegression().fit(preds, labels)

        # 測試時套用 regression 校正
        angle_errors = []
        for d in test_set:
            pred = model(
                d['person_idx'], d['face'], d['right_eye'], d['left_eye']
            ).squeeze(0).cpu().numpy()
            corrected = reg.predict([pred])[0]
            angle = calc_angle_error(
                torch.tensor(corrected).unsqueeze(0).to(device),
                d['label'].unsqueeze(0)
            )
            angle_errors.append(angle.item())

        mean_err = np.mean(angle_errors)
        print(f"📐 Linear Calibration (S={S}): {mean_err:.2f} degrees")
        return mean_err


In [7]:
def simulate_calibration(model, dataset, S=1, device='cuda'):
    """
    模擬 S 張圖當校正資料，對剩下資料測試。
    模型 forward 接收格式：FinalModel(person_idx, full_face, right_eye, left_eye)
    """
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        all_data = []
        for i in range(len(dataset)):
            item = dataset[i]
            data = {
                'left_eye': item['left_eye_image'].unsqueeze(0).to(device),
                'right_eye': item['right_eye_image'].unsqueeze(0).to(device),
                'face': item['full_face_image'].unsqueeze(0).to(device),
                'label': torch.tensor(
                    [float(item['gaze_pitch']), float(item['gaze_yaw'])],
                    dtype=torch.float32
                ).to(device),
                'person_idx': torch.tensor([[item['person_idx']]]).to(device)  # shape [1, 1]
            }
            all_data.append(data)

        # Step 1：選 S 張圖當校正資料
        indices = np.random.choice(len(all_data), size=S, replace=False)
        calibration_set = [all_data[i] for i in indices]
        test_set = [all_data[i] for i in range(len(all_data)) if i not in indices]

        # Step 2：計算 bias
        cal_outputs, cal_labels = [], []
        for d in calibration_set:
            pred = model(
                d['person_idx'],
                d['face'],
                d['right_eye'],
                d['left_eye']
            ).squeeze(0)
            cal_outputs.append(pred)
            cal_labels.append(d['label'])

        bias = torch.stack(cal_labels).mean(dim=0) - torch.stack(cal_outputs).mean(dim=0)

        # Step 3：套用 bias 做測試
        angle_errors = []
        for d in test_set:
            pred = model(
                d['person_idx'],
                d['face'],
                d['right_eye'],
                d['left_eye']
            ).squeeze(0) + bias

            angle = calc_angle_error(pred.unsqueeze(0), d['label'].unsqueeze(0))
            angle_errors.append(angle.item())

        mean_err = np.mean(angle_errors)
        print(f"📐 Calibration (S={S}): {mean_err:.2f} degrees")
        return mean_err

In [8]:
if __name__ == '__main__':

    ###非變動區塊
    k = [9,128]
    adjust_slope = False
    ###對應作者表格grid calibration,如果是random就填true
    grid_calibration_samples=True
    batch_size =32
    path_to_data = './data/mpiifacegaze_preprocessed'
    
    
    ####變動區塊
    path_to_checkpoints = './saved_models/p00/p00_best-v46.ckpt'
    person_idx = 1
    validate_on_person = 0 
    
    seed_everything(42)

    print(f"{path_to_checkpoints}")
    
    model = Model.load_from_checkpoint(path_to_checkpoints, k=[9, 128], adjust_slope=False, grid_calibration_samples=True)

    trainer = Trainer(
        gpus=1,
        benchmark=True,
    )

    _, _, test_dataloader = get_dataloaders(
        path_to_data, 
        validate_on_person, 
        person_idx, 
        batch_size   
    )
     
    trainer.test(model, test_dataloader)

    ####################拿出 test set（單人完整資料）
    test_dataset = test_dataloader.dataset
    #######################


    #####STGC呼叫##########
    #evaluate_with_eq3_repeat(model, test_dataset, s=1, repeat_times=10)
    #evaluate_with_eq3_repeat(model, test_dataset, s=5, repeat_times=10)
    #evaluate_with_eq3_repeat(model, test_dataset, s=9, repeat_times=10)
    #evaluate_with_eq3_repeat(model, test_dataset, s=16, repeat_times=10)

    ######MTGC
    #evaluate_with_eq3_repeat(model, test_dataset, s=1, repeat_times=10, use_mgtc=True)
    #evaluate_with_eq3_repeat(model, test_dataset, s=5, repeat_times=10, use_mgtc=True)
    #evaluate_with_eq3_repeat(model, test_dataset, s=9, repeat_times=10, use_mgtc=True)
    #evaluate_with_eq3_repeat(model, test_dataset, s=16, repeat_times=10, use_mgtc=True)
    #evaluate_with_eq3_repeat(model, test_dataset, s=32, repeat_times=10, use_mgtc=True)
    #evaluate_with_eq3_repeat(model, test_dataset, s=64, repeat_times=10, use_mgtc=True)
    #evaluate_with_eq3_repeat(model, test_dataset, s=128, repeat_times=10, use_mgtc=True)



    #evaluate_with_eq3(model, test_dataset, s=1, target_pitch=0.0, target_yaw=0.0)
    #evaluate_with_eq3(model, test_dataset, s=5, target_pitch=0.0, target_yaw=0.0)
    #evaluate_with_eq3(model, test_dataset, s=9, target_pitch=0.0, target_yaw=0.0)
    #evaluate_with_eq3(model, test_dataset, s=16, target_pitch=0.0, target_yaw=0.0)


Global seed set to 42


./saved_models/p00/p00_best-v46.ckpt


C:\Users\User\anaconda3\envs\gazegpu\lib\site-packages\pytorch_lightning\loops\utilities.py:92: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


train on persons [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
valid on person 0
test on person 1
len(dataset_train) 60784
len(dataset_valid) 2927


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


len(dataset_test) 2904
Testing DataLoader 0: 100%|██████████| 181/181 [00:26<00:00,  6.78it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
             Test metric                         DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test/offset(k=0)/angular_error              3.207423686981201
        test/offset(k=0)/loss                0.0019713188521564007
test/offset(k=128)/mean_angular_error         2.4511219635009764
test/offset(k=128)/std_angular_error         0.016094765126522557
 test/offset(k=9)/mean_angular_error           2.528395970821381
 test/offset(k=9)/std_angular_error           0.11120549839860427
  test/offset(k=all)/angular_error             2.450024366378784
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
